In [1]:
import pandas as pd
import numpy as np
import numpy.linalg as linalg
import statsmodels.api as sm
from pandas.tseries.offsets import MonthEnd
import mysql.connector
from mysql.connector import errorcode
from sqlalchemy import create_engine
import utils as ut
from statsmodels.tsa.api import VAR

C:\Users\arizonazhang\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\arizonazhang\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
ut.load_data_sql('us','industry_group',start_dt="2005-01")

(col         .TR15GSPAU  .TR15GSPBK  .TR15GSPCS  .TR15GSPDF  .TR15GSPE  \
 date                                                                    
 2006-04-07   -0.024512    0.003222   -0.004717    0.008300   0.016111   
 2006-04-14   -0.018906   -0.001891   -0.003317   -0.001809   0.004380   
 2006-04-21    0.039543    0.012827    0.022588    0.019194   0.070952   
 2006-04-28   -0.002285    0.037474    0.009439    0.010897  -0.038551   
 2006-05-05    0.022212    0.002525    0.005571    0.009810   0.027665   
 ...                ...         ...         ...         ...        ...   
 2022-03-04   -0.004272   -0.076510    0.033929   -0.030502   0.095090   
 2022-03-11   -0.045182   -0.021401   -0.029302   -0.021402   0.021143   
 2022-03-18    0.121561    0.057165    0.074124    0.078065  -0.034175   
 2022-03-25    0.082549    0.011944   -0.004994    0.011201   0.077063   
 2022-04-01    0.058121   -0.061027    0.019817   -0.019962  -0.021216   
 
 col         .TR15GSPFBT  .TR15GSPFD

In [9]:
import warnings
from utils import *
import pandas as pd
import numpy as np
from cvxopt import matrix, solvers  # quadratic programming


def cal_weights(cal_mean_var, gamma_op, model_log=False):
    """
    calculate the allocation based on the cal_mean_var function given and gamma_op specified
    return a dataframe containing weightings of all time points
    """
    weights = {}
    if model_log:
        evec = {}

    # "weeks" stands for the rebalancing weeks
    weeks = pd.date_range(start=df_sectors.index[0], end=df_sectors.index[-1], freq="W-FRI")
    reb = len(weeks[weeks < df_sectors.index[window - 1]])

    for t in range(T - window):
        # rebalancing week
        row_index = df_sectors.index[t + window].strftime("%Y-%m-%d")
        if reb < len(weeks) and df_sectors.index[t + window - 1] <= weeks[reb] and df_sectors.index[t + window] > weeks[
            reb]:

            # select the calculation time horizon (last 260 weeks including holidays)
            idx_range = (df_sectors.index <= weeks[reb]) & (df_sectors.index >= weeks[reb - window + 1])
            df_window = df_sectors[idx_range].dropna(how='any', axis=1)
            p_window = len(df_window.columns) - n

            # calculate E and V using the method specified
            if cal_mean_var.__name__ == "factorx":
                exog_t = exogs[(np.cumsum(idx_range) == np.sum(idx_range)) & (~idx_range)].values[0]
                exog_t1 = (np.sum(exogs[idx_range].iloc[-51:, ]) + exog_t) / 52
                exog_future = np.reshape(np.concatenate([exog_t, exog_t1, exog_t1, exog_t1]), (4, exogs.shape[1]))

                Evec, Vmat = cal_mean_var(df_window, exog=exogs[idx_range], exog_future=exog_future, p=p_window)
            else:
                Evec, Vmat = cal_mean_var(df_window, p=p_window)
            Vmat = Vmat.astype('float')  # must convert otherwise the package gives a warning
            Evec = Evec.astype('float')
            if t == 272:
                print(Vmat)
                print(Evec)

            # optimization
            solvers.options['show_progress'] = False  # do not show optimization output
            solvers.options['abstol'] = 0.1 ** -6
            solvers.options['reltol'] = 0.1 ** -5
            Q = gamma_op * matrix([list(Vmat[i, :]) for i in range(p_window)])
            q = -1 * matrix(list(Evec))
            G = matrix([[0.0] * i + [-1.0] + [0.0] * (p_window - i - 1) for i in range(p_window)])
            h = matrix([0.0] * p_window)
            A = matrix([1.0] * p_window, (1, p_window))
            b = matrix(1.0)

            try:
                sol = solvers.qp(Q, q, G, h, A, b)
                weights[row_index] = dict(zip(list(df_window.columns), list(sol['x'])))
            except:
                print("cannot find solution at {}".format(t + window))
                weights[row_index] = dict(zip(list(df_window.columns), list(np.ones(p_window) / p_window)))

            # save records if model log is TRUE
            if model_log:
                evec[row_index] = dict(zip(list(df_window.columns), list(Evec)))
            reb += 4

        # non-rebalancing week
        else:
            row_index_pre = df_sectors.index[t + window - 1].strftime("%Y-%m-%d")
            new_weights = pd.Series(weights[row_index_pre])*(1 + df_sectors.iloc[t + window - 1, :p])
            new_weights = new_weights / np.sum(new_weights)
            weights[row_index] = new_weights.to_dict()


    # save weights in dataframe
    weights = pd.DataFrame.from_dict(weights, orient="index")
    if model_log:
        evec = pd.DataFrame.from_dict(evec, orient='index')
        evec.to_csv(r".\regre\evec_{}_{}.csv".format(mkt, cal_mean_var.__name__))
    return weights


def cal_ew(df_sectors):
    """
    calculate the weekly weighting of an equally weighted portfolio, rebalanced monthly
    """
    weights = {}
    weeks = pd.date_range(start=df_sectors.index[0], end=df_sectors.index[-1], freq="W-FRI", closed="left")
    reb = len(weeks[weeks < df_sectors.index[window - 1]])
    for t in range(T - window):
        row_index = df_sectors.index[t + window].strftime("%Y-%m-%d")
        if reb < len(weeks) and df_sectors.index[t + window - 1] <= weeks[reb] and df_sectors.index[t + window] > weeks[reb]:
            idx_range = (df_sectors.index <= weeks[reb]) & (df_sectors.index >= weeks[reb - window + 1])
            df_window = df_sectors[idx_range].dropna(how='any', axis=1) # TODO: could optimize
            p_window = len(df_window.columns) - n

            weights[row_index] = dict(zip(list(df_window.columns), list(np.ones(p_window) / p_window)))
            reb += 4
        else:
            row_index_pre = df_sectors.index[t + window - 1].strftime("%Y-%m-%d")
            new_weights = pd.Series(weights[row_index_pre]) * (1 + df_sectors.iloc[t + window - 1, :p])
            new_weights = new_weights / np.sum(new_weights)
            weights[row_index] = new_weights.to_dict()

    weights = pd.DataFrame.from_dict(weights, orient="index")
    return weights

if __name__ == "__main__":
    # filter out future warning for pd.concat
    warnings.simplefilter(action='ignore', category=FutureWarning)

    # define parameters
    # sector_indices = ("hsci", "csi", "dj")
    # markets = ("hk", "cn", "us")
    # level = "sector"
    total_ret = False

    markets = ("hk",)
    level = "industry_group"

    window = 156
    func_list = [est_input_capm, est_input_factor, est_input_hist,
                 est_input_factorx]  # list of methods (est_input_capm_old, est_input_factor_old)
    for i in range(len(markets)):
        for est_input in func_list:
#         for est_input in [est_input_capm]:
            ls_weight = []
            ls_return = []
            mkt = markets[i]

            # get from local files
            # df_sectors, exogs, p, n, T = load_data(mkt, level)
            # or, get from mysql database
            df_sectors, exogs, p, n, T = load_data_sql(mkt, level)

            for gamma_op in [2, 4, 8]:
                weights = cal_weights(est_input, gamma_op,
                                      model_log=True if gamma_op == 2 else False)  # save model outputs when gamma=2 (for checking)
                weights["gamma_op"] = gamma_op
                ls_weight.append(weights)

                # calculate weekly return of the allocation
                returns = cal_return(df_sectors.iloc[window:, :p], weights.iloc[:, :p])
                returns["gamma_op"] = gamma_op
                ls_return.append(returns)

            # calculate the return of equally-weighted portfolio - for benchmarking
            equal = cal_ew(df_sectors)
            equal_return = cal_return(df_sectors.iloc[window:, :p], equal)

            if total_ret: # suffix for test cases
                suffix = "t"
            else: # default suffix
                suffix = "e"

            try:
                w_suffix = "3" if window < 260 else "" # add suffix in file name to indicate 3y window

                pd.concat(ls_weight).to_csv(rf".\weights\{mkt}_{level}_{est_input.__name__ + suffix}{w_suffix}.csv")
                pd.concat(ls_return).to_csv(rf".\performances\{mkt}_{level}_{est_input.__name__ + suffix}{w_suffix}.csv")

                equal.to_csv(rf".\weights\{mkt}_{level}_ew{w_suffix}.csv")
                equal_return.to_csv(rf".\performances\{mkt}_{level}_ew{w_suffix}.csv")

            except PermissionError:
                print("please close data outputs.")
            else:
                print(f"Output successful, {mkt}_{level}_{est_input.__name__ + suffix}{w_suffix}")





[[1.78190883e-03 4.48647641e-05 1.62295012e-05 2.21544330e-05
  4.33007297e-05 2.65026695e-05 8.04299453e-06 1.28456878e-05
  1.85460192e-05 3.41916282e-05 2.90645161e-05 2.29538587e-05
  4.79222956e-05 1.12234762e-05 1.22830414e-05 2.29740679e-05
  2.59008332e-05]
 [4.48647641e-05 4.40685264e-03 4.37918268e-05 5.97789841e-05
  1.16837729e-04 7.15117675e-05 2.17022951e-05 3.46613325e-05
  5.00424537e-05 9.22587728e-05 7.84243608e-05 6.19360629e-05
  1.29308033e-04 3.02841426e-05 3.31431520e-05 6.19905930e-05
  6.98878414e-05]
 [1.62295012e-05 4.37918268e-05 8.31913060e-04 2.16246115e-05
  4.22651962e-05 2.58688603e-05 7.85064695e-06 1.25384842e-05
  1.81024926e-05 3.33739381e-05 2.83694405e-05 2.24049190e-05
  4.67762377e-05 1.09550677e-05 1.19892935e-05 2.24246448e-05
  2.52814169e-05]
 [2.21544330e-05 5.97789841e-05 2.16246115e-05 9.05693983e-04
  5.76950239e-05 3.53128495e-05 1.07166961e-05 1.71159301e-05
  2.47112006e-05 4.55578190e-05 3.87263208e-05 3.05843212e-05
  6.38529190e-05

[[1.78190883e-03 4.48647641e-05 1.62295012e-05 2.21544330e-05
  4.33007297e-05 2.65026695e-05 8.04299453e-06 1.28456878e-05
  1.85460192e-05 3.41916282e-05 2.90645161e-05 2.29538587e-05
  4.79222956e-05 1.12234762e-05 1.22830414e-05 2.29740679e-05
  2.59008332e-05]
 [4.48647641e-05 4.40685264e-03 4.37918268e-05 5.97789841e-05
  1.16837729e-04 7.15117675e-05 2.17022951e-05 3.46613325e-05
  5.00424537e-05 9.22587728e-05 7.84243608e-05 6.19360629e-05
  1.29308033e-04 3.02841426e-05 3.31431520e-05 6.19905930e-05
  6.98878414e-05]
 [1.62295012e-05 4.37918268e-05 8.31913060e-04 2.16246115e-05
  4.22651962e-05 2.58688603e-05 7.85064695e-06 1.25384842e-05
  1.81024926e-05 3.33739381e-05 2.83694405e-05 2.24049190e-05
  4.67762377e-05 1.09550677e-05 1.19892935e-05 2.24246448e-05
  2.52814169e-05]
 [2.21544330e-05 5.97789841e-05 2.16246115e-05 9.05693983e-04
  5.76950239e-05 3.53128495e-05 1.07166961e-05 1.71159301e-05
  2.47112006e-05 4.55578190e-05 3.87263208e-05 3.05843212e-05
  6.38529190e-05

[[ 1.79546578e-03  6.75337135e-05 -3.54249515e-05  3.69927837e-05
  -2.11454051e-05 -1.07726104e-05 -4.11785083e-05  1.75737696e-05
   3.91526408e-05 -1.12181818e-06 -6.15476299e-05 -7.09206884e-06
  -2.16895746e-05  1.09852483e-05 -5.06655321e-05  3.00836211e-05
   2.13513822e-05]
 [ 6.75337135e-05  4.45425173e-03  1.48974936e-04  1.42565016e-04
   2.59870957e-04  2.23740009e-04  6.68428152e-05  6.71634120e-05
   1.13367631e-04  1.59531582e-04  1.61152692e-04  1.65553545e-04
   2.56981859e-04  4.50666326e-05  1.39380478e-04  1.32954583e-04
   1.79218643e-04]
 [-3.54249515e-05  1.48974936e-04  8.41521647e-04  9.05977087e-05
   1.59380355e-04  1.01208168e-04  1.17853132e-04  5.67075231e-05
   5.86631371e-05  1.03304483e-04  9.52060706e-05  7.83674153e-05
   1.86616647e-04  4.16312709e-05  1.30298613e-04  9.58400847e-05
   1.13545588e-04]
 [ 3.69927837e-05  1.42565016e-04  9.05977087e-05  9.13707438e-04
   1.11193979e-04  6.46559374e-05  8.35405214e-05  6.02676241e-05
   6.33867141e-05  

Output successful, hk_industry_group_factore3
[[0.00178282 0.00081922 0.00039404 0.00047538 0.0006169  0.00065124
  0.00050244 0.00036807 0.0004197  0.00058156 0.00047487 0.00049992
  0.00049787 0.00017715 0.00050303 0.00054277 0.00061035]
 [0.00081922 0.00441351 0.00071269 0.0007623  0.00178284 0.00183481
  0.00145638 0.00049435 0.00051518 0.00095098 0.00099652 0.00102754
  0.00163794 0.00027582 0.00135014 0.0011359  0.00103906]
 [0.00039404 0.00071269 0.00083278 0.00061899 0.00069562 0.00061927
  0.0008579  0.00037817 0.00045843 0.00051556 0.00064958 0.00052615
  0.00080262 0.0002789  0.00074745 0.00080179 0.00066147]
 [0.00047538 0.0007623  0.00061899 0.00090732 0.00070184 0.00071161
  0.00095039 0.00047171 0.00056513 0.00056749 0.00068653 0.00063261
  0.00084984 0.00041131 0.00083512 0.00094917 0.00073399]
 [0.0006169  0.00178284 0.00069562 0.00070184 0.00293108 0.00187126
  0.00132208 0.00044162 0.00034058 0.0011673  0.00083547 0.00093774
  0.00155111 0.00024797 0.00100143 0.00090

Output successful, hk_industry_group_histe3
[[ 1.80070455e-03  6.87462880e-05 -3.67226536e-05  3.75435979e-05
  -2.30454233e-05 -1.22314542e-05 -4.41866105e-05  1.75441383e-05
   4.00706233e-05 -2.72362081e-06 -6.53750763e-05 -9.03631021e-06
  -2.37176580e-05  1.13208891e-05 -5.20401931e-05  3.09877627e-05
   2.07062002e-05]
 [ 6.87462880e-05  4.48098273e-03  1.57911140e-04  1.49436238e-04
   2.73723612e-04  2.31447764e-04  6.61344321e-05  7.03039975e-05
   1.18847240e-04  1.68329986e-04  1.69983831e-04  1.70914196e-04
   2.70931680e-04  4.88479407e-05  1.47366011e-04  1.40888930e-04
   1.87369740e-04]
 [-3.67226536e-05  1.57911140e-04  8.49177514e-04  9.64510490e-05
   1.69611727e-04  1.05859871e-04  1.22165181e-04  6.03600992e-05
   6.22186390e-05  1.10623108e-04  1.02261740e-04  8.27778215e-05
   1.98514304e-04  4.52005787e-05  1.37543025e-04  1.02163719e-04
   1.20911809e-04]
 [ 3.75435979e-05  1.49436238e-04  9.64510490e-05  9.19107428e-04
   1.18090248e-04  6.68080304e-05  8.5832

[[ 1.80070455e-03  6.87462880e-05 -3.67226536e-05  3.75435979e-05
  -2.30454233e-05 -1.22314542e-05 -4.41866105e-05  1.75441383e-05
   4.00706233e-05 -2.72362081e-06 -6.53750763e-05 -9.03631021e-06
  -2.37176580e-05  1.13208891e-05 -5.20401931e-05  3.09877627e-05
   2.07062002e-05]
 [ 6.87462880e-05  4.48098273e-03  1.57911140e-04  1.49436238e-04
   2.73723612e-04  2.31447764e-04  6.61344321e-05  7.03039975e-05
   1.18847240e-04  1.68329986e-04  1.69983831e-04  1.70914196e-04
   2.70931680e-04  4.88479407e-05  1.47366011e-04  1.40888930e-04
   1.87369740e-04]
 [-3.67226536e-05  1.57911140e-04  8.49177514e-04  9.64510490e-05
   1.69611727e-04  1.05859871e-04  1.22165181e-04  6.03600992e-05
   6.22186390e-05  1.10623108e-04  1.02261740e-04  8.27778215e-05
   1.98514304e-04  4.52005787e-05  1.37543025e-04  1.02163719e-04
   1.20911809e-04]
 [ 3.75435979e-05  1.49436238e-04  9.64510490e-05  9.19107428e-04
   1.18090248e-04  6.68080304e-05  8.58321059e-05  6.34646282e-05
   6.66292798e-05  